Thoughts:
1. extract speech as text, store in an ordered list (or other data structure), then analyze, and use context clues from text to identify speaker
2. I'm writing some initial code to extract data from data.json, print it out (yeah yeah i can read it directly too)
3. decided to create an ordered list of dictionaries to store the text and potentially useful information. The "role" of the speaker is a placeholder, currently empty
4. have never used Spacy before but prompting chat GPT says it's the best option. installing it
5. it took me a while to get spacy to work because my environments were mixed up! (conda vs homebrew)had to do some install/reinstall of jupyter notebook, should have just used VScode...
7. using spacy to tokenize, lemmatize, etc. my code has a basic training data set, and then using TfidVectorizer to convert text data into features, then I train a naive bayes classifier to distinguish between providers and patients from the text, and predict those roles. 
8. Ok! this doesn't work perfectly ({'text': 'No, none at all.', 'original_label': 'A', 'role': 'provider'}
 is labelled incorrectly. Ideally, I would want to get a large amount of training data and split it into my validation, testing, and training sets. I could comb existing transcripts (ex; i2b2) of patient/provider interactions from online, or make GPT generate them for me (the first one is preferred). I'm currently using Naive Baiyes classifier from scikit-learn, so adding the training-testing split would be a first distinction to make moving forward. Then I would focus on tuning the hyperparameters to optimize performance. I could also switch to a different model. I could improve feature engineering as well. 


In [ ]:
import json

with open('data.json') as f:
    data = json.load(f)

data

for chunk in data['chunks']:
    for utterance in chunk['utterances']:
        label = utterance['speaker']
        print(f"{label}: {utterance['text']}")


In [ ]:
utterances_list = []
for chunk in data['chunks']:
    for utterance in chunk['utterances']:
        utterances_list.append({
            'text': utterance['text'],
            'original_label': utterance['speaker'],
            'role': None  # Placeholder for the inferred role
        })
for utterance in utterances_list:
    print(utterance)

In [ ]:
import sys
print(sys.executable)
print(sys.version)


In [3]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

# Ensure spaCy and scikit-learn are installed
!pip install spacy scikit-learn
!python -m spacy download en_core_web_sm

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Enhanced training data
training_data = [
    ("How are you feeling today?", "provider"),
    ("I'm feeling better, thank you.", "patient"),
    ("Have you been taking your medication?", "provider"),
    ("Yes, every day.", "patient"),
    ("Any side effects?", "provider"),
    ("No, none at all.", "patient"),
    ("That's too bad. Let's get you fixed up.", "provider"),
    ("And what about your arms?", "provider"),
    ("They are still kind of hurting.", "patient"),
    ("Please describe your symptoms.", "provider"),
    ("I have a persistent cough.", "patient"),
    ("When did the symptoms start?", "provider"),
    ("They started about a week ago.", "patient"),
    ("Have you taken any medication for this?", "provider"),
    ("Yes, I've been taking cough syrup.", "patient"),
    ("Are you experiencing any side effects?", "provider"),
    ("No, I haven't noticed any side effects.", "patient"),
    ("Make sure to get plenty of rest.", "provider"),
    ("Thank you, I will.", "patient"),
    ("Do you have any allergies?", "provider"),
    ("I'm allergic to penicillin.", "patient"),
    ("We'll prescribe a different antibiotic.", "provider"),
    ("Thank you, doctor.", "patient"),
    ("Can you rate your pain on a scale of 1 to 10?", "provider"),
    ("It's around a 7.", "patient"),
    ("How is your appetite?", "provider"),
    ("I've not been very hungry lately.", "patient"),
    ("Any changes in your sleep pattern?", "provider"),
    ("Yes, I'm finding it hard to fall asleep.", "patient"),
    ("Are you experiencing any other symptoms?", "provider"),
    ("No, that's all.", "patient"),
    ("Do you have any chronic conditions?", "provider"),
    ("I have asthma.", "patient"),
]

# Split training data into texts and labels
texts, labels = zip(*training_data)

# Function to preprocess text using spaCy
def spacy_tokenizer(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

# Preprocess the texts
texts = [spacy_tokenizer(text) for text in texts]

# Create a pipeline that combines TfidfVectorizer with Naive Bayes classifier
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Train the model
model.fit(texts, labels)

# Function to predict the role using the trained model
def predict_role(text):
    processed_text = spacy_tokenizer(text)
    return model.predict([processed_text])[0]

# Load the data
import json
with open('data.json') as f:
    data = json.load(f)

# Initialize an empty list to store utterances
utterances_list = []

# Extract data and store in the list
for chunk in data['chunks']:
    for utterance in chunk['utterances']:
        utterances_list.append({
            'text': utterance['text'],
            'original_label': utterance['speaker'],
            'role': None  # Placeholder for the inferred role
        })

# Predict roles for the utterances in the data
for utterance in utterances_list:
    utterance['role'] = predict_role(utterance['text'])

# Display the utterances with inferred roles
for utterance in utterances_list:
    print(utterance)

# Evaluate the model with the training data
predicted_labels = [predict_role(text) for text in texts]
print(classification_report(labels, predicted_labels))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 4.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
{'text': 'Hello, how are you feeling today?', 'original_label': 'A', 'role': 'provider'}
{'text': "I'm feeling better, thank you.", 'original_label': 'B', 'role': 'patient'}
{'text': 'Have you been taking your medication regularly?', 'original_label': 'A', 'role': 'provider'}
{'text': "Yes, I've been taking it every day.", 'original_label': 'A', 'role': 'patient'}
{'text': 'Great to hear! Any side effects?', 'original_label': 'B', 'role': 'provider'}
{'text': 'No, none at all.', 'original_label': 'A', 'role': 'provider'}
{'text': 'And what about your arms?', 'original_label': 'A', 'role': 'provider'}
{'text': 'They are still kind of hurting.', 'original_label': 'B', 'role': 'patient'}
{'text': "That's too bad. Let's get you fixed up.", 'original_label': 'A', 'role': 'provider'}
              precis